In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import pandas as pd
from pathlib import Path

sys.path.append("..")

In [3]:
import tensorflow as tf
import torch

from librep.transforms import TSNE
from librep.transforms import UMAP
from librep.datasets.multimodal import TransformMultiModalDataset, ArrayMultiModalDataset
from librep.metrics.dimred_evaluator import DimensionalityReductionQualityReport

2022-09-20 04:33:46.097264: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-20 04:33:46.097283: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
dataset = tf.keras.datasets.mnist.load_data(path="mnist.npz")
(train_x, train_y), (test_x, test_y) = dataset

In [5]:
train_x_reordered = train_x.reshape((60000,-1))
print('TRAIN_X_REORDERED', train_x_reordered.shape)

test_x_reordered = test_x.reshape((10000,-1))
print('TEST_X_REORDERED', test_x_reordered.shape)


TRAIN_X_REORDERED (60000, 784)
TEST_X_REORDERED (10000, 784)


In [6]:
mnist_dataset_train = ArrayMultiModalDataset(X=train_x_reordered, y=train_y, window_slices=[(0, 28*28)], 
                                             window_names=["px"])
mnist_dataset_test = ArrayMultiModalDataset(X=test_x_reordered, y=test_y, window_slices=[(0, 28*28)], 
                                             window_names=["px"])

In [7]:
from librep.transforms.topo_ae import TopologicalDimensionalityReduction
from librep.estimators.ae.torch.models.topological_ae.topological_ae import TopologicallyRegularizedAutoencoder

In [8]:
# topo_transformer = TopologicallyRegularizedAutoencoder(autoencoder_model='DeepAE')
# optimizer = torch.optim.Adam(topo_transformer.parameters(), lr=1e-3, weight_decay=1e-5)
# autoencoder_model='DeepAE'
# data_loader = torch.utils.data.DataLoader(dataset=train_x, batch_size=128, shuffle=True)

Using python to compute signatures


In [15]:
transform_topoae = TopologicalDimensionalityReduction(ae_model='DeepAE')
transform_topoae.fit(mnist_dataset_train.X)

Topological DeepAE
Using python to compute signatures
Epoch:1, Loss:6838.5918
Epoch:2, Loss:7692.8037
Epoch:3, Loss:7542.8457
Epoch:4, Loss:7693.5801
Epoch:5, Loss:7557.3994
Epoch:6, Loss:6822.8433
Epoch:7, Loss:7471.2964
Epoch:8, Loss:7664.0977
Epoch:9, Loss:7590.1279
Epoch:10, Loss:7589.0073
Epoch:11, Loss:7383.7358
Epoch:12, Loss:6936.3936
Epoch:13, Loss:7674.0063


In [13]:
test_applied_topoae = transform_topoae.transform(mnist_dataset_test.X)

In [14]:
print(test_applied_topoae.shape)

(10000, 2)


In [20]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=128)
metrics_train_applied_topoae = metrics_reporter.evaluate([mnist_dataset_test, test_applied_topoae])
print(metrics_train_applied_topoae)

{'residual variance (pearson)': 0.9246437458825869, 'residual variance (spearman)': 0.9556982407321778, 'trustworthiness': 0.7075530823070193, 'continuity': 0.7617092220930803, 'co k nearest neighbor size': 0.30087194629517466, 'local continuity meta criterion': 0.17388781931104766, 'local property': 0.21597443066450644, 'global property': 0.6328359673144206}


In [21]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=500)
metrics_train_applied_topoae = metrics_reporter.evaluate([mnist_dataset_test, test_applied_topoae])
print(metrics_train_applied_topoae)

{'residual variance (pearson)': 0.9584142086930323, 'residual variance (spearman)': 0.9701272731080088, 'trustworthiness': 0.6858645917748105, 'continuity': 0.7945079026965339, 'co k nearest neighbor size': 0.16332665330661322, 'local continuity meta criterion': 0.13119813925038837, 'local property': 0.19839350218310767, 'global property': 0.6248983029187553}


In [22]:
metrics_reporter = DimensionalityReductionQualityReport(sampling_threshold=1000)
metrics_train_applied_topoae = metrics_reporter.evaluate([mnist_dataset_test, test_applied_topoae])
print(metrics_train_applied_topoae)

{'residual variance (pearson)': 0.9656163132223349, 'residual variance (spearman)': 0.9724199868883134, 'trustworthiness': 0.6819253129019152, 'continuity': 0.8131425055664911, 'co k nearest neighbor size': 0.11419753086419752, 'local continuity meta criterion': 0.09816546673594102, 'local property': 0.1954404389695359, 'global property': 0.625258618900966}
